## learning kernels for the GLM example. 

we optimize kernels such that
$ K(x_n, x_0) p(\theta_n) / \tilde{p}(\theta_n) \approx 1$. 

Spoiler:
starts to work.


In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

seeds = np.arange(90, 110)

batch_specifier = '_cp'
fit_specifier = '_5dim'
if batch_specifier == '_bp':
    smoothing_prior = utils.smoothing_prior_highVar
elif batch_specifier == '_cp':
    smoothing_prior = utils.smoothing_prior   

duration = 300

len_filter, a = 4, 2.
true_params, labels_params = utils.obs_params(len_filter, a)

def init_g(seed):
    
    s = GLMStats(n_summary=len_filter+1)
    p = smoothing_prior(n_params=len_filter+1, seed=seed)
    m = GLM(len_filter=len_filter, duration=duration, seed=seed)
    g = dg.Default(summary=s, model=m, prior=p)    
    
    return g
    
n_train=3000
n_rounds=5
minibatch=100 
epochs=200   
round_cl=3

minibatch_cbk=n_train
epochs_cbk=10000

n_hiddens = [10,10]
convert_to_T = 3
pilot_samples = 1000
svi=True
reg_lambda=0.01
prior_norm=False
cbk_feature_layer = 4

for seed in seeds:
    
    obs = utils.obs_data(true_params, seed=seed, duration=duration)
    obs_stats = utils.obs_stats(true_params, seed=seed, duration=duration)

    
    sam = utils.pg_mcmc(true_params, obs, duration=duration, 
                        prior_dist=smoothing_prior(n_params=len_filter+1, seed=seed))
    np.savez('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz', sam)
    
    pe = dd.Gaussian(m=sam.mean(axis=1), S=np.cov(sam))


    res = infer.SNPE(generator=init_g(seed), 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)
    logs, tds, posteriors = res.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    minibatch_cbk=minibatch_cbk,
                                    epochs_cbk=epochs_cbk,                                        
                                    round_cl=round_cl, 
                                    kernel_loss=None)

    res_ki = infer.SNPE(generator=init_g(seed), 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)
    logs_ki, tds_ki, posteriors_ki = res_ki.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    minibatch_cbk=minibatch_cbk,
                                    epochs_cbk=epochs_cbk,                                        
                                    round_cl=round_cl, 
                                    cbk_feature_layer=0,
                                    kernel_loss='x_kl')

    res_kh = infer.SNPE(generator=init_g(seed), 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)
    logs_kh, tds_kh, posteriors_kh = res_kh.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    minibatch_cbk=minibatch_cbk,
                                    epochs_cbk=epochs_cbk,                                        
                                    round_cl=round_cl, 
                                    cbk_feature_layer=cbk_feature_layer,
                                    kernel_loss='x_kl')
    
    np.save('check_higherOrder_kernels' + batch_specifier + fit_specifier + '_d' + str(duration) + '_' + str(seed),
            {'seed': seed,
             'duration' : duration, 
             'n_train' : n_train,
             'n_rounds' : n_rounds,
             'minibatch' : minibatch,
             'epochs' : minibatch,

             'epochs_cbk' : epochs_cbk,
             'minibatch_cbk' : minibatch_cbk,
             
             'n_hiddens' : n_hiddens, 
             'convert_to_T' : convert_to_T, 
             'pilot_samples' : pilot_samples,
             'svi' : svi,
             'reg_lambda': reg_lambda,
             'prior_norm': prior_norm,             
             'round_cl' : round_cl,     
             
             'obs_stats' : obs_stats,
             'true_params' : true_params,
             
             'g' : init_g(seed), 
             'pe' : pe, 

             'cbk_feature_layer' : cbk_feature_layer,
             'logs' : logs, 
             'logs_ki' : logs_ki, 
             'logs_kh' : logs_kh, 
             'tds' : tds,
             'tds_ki' : tds_ki,
             'tds_kh' : tds_kh,
             'posteriors' : posteriors,
             'posteriors_ki' : posteriors_ki,
             'posteriors_kh' : posteriors_kh
             })    

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

seeds = np.arange(90, 110)
duration = 500


batch_specifier = '_bp'
fit_specifier = '_pilot_test_05'


for seed in seeds:
    true_params, labels_params = utils.obs_params(a=a_)
    obs = utils.obs_data(true_params, seed=seed, duration = duration)
    obs_stats = utils.obs_stats(true_params, seed=seed, duration = duration)

    rerun=False
    
    try:
        assert rerun == False, 'rerun requested'
        sam = np.load('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz')['arr_0']
    except:
        sam = utils.pg_mcmc(true_params, obs, duration = duration)
        np.savez('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz', sam)

    n_train = 5000
    n_rounds = 4
    minibatch = 100
    epochs = 500
    round_cl = 4

    epochs_cbk = 5000
    minibatch_cbk = n_train
    
    n_hiddens = [50] 
    convert_to_T = None
    pilot_samples = 1000
    svi = True
    reg_lambda = 0.01
    prior_norm = False
    
    if batch_specifier == '_bp':
        smoothing_prior = utils.smoothing_prior_highVar
    elif batch_specifier == '_cp':
        smoothing_prior = utils.smoothing_prior    
    
    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)

    res = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs, tds, posteriors = res.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss=None)

    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)
    res_k = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs_k, tds_k, posteriors_k = res_k.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss='x_kl')

    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)
    res_k2 = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs_k2, tds_k2, posteriors_k2 = res_k2.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss='ess')
    
    np.save('check_kernels' + batch_specifier + fit_specifier + '_d' + str(duration) + '_' + str(seed),
            {'seed': seed,
             'duration' : duration, 
             'n_train' : n_train,
             'n_rounds' : n_rounds,
             'minibatch' : minibatch,
             'epochs' : minibatch,

             'epochs_cbk' : epochs_cbk,
             'minibatch_cbk' : minibatch_cbk,
             
             'n_hiddens' : n_hiddens, 
             'convert_to_T' : convert_to_T, 
             'pilot_samples' : pilot_samples,
             'svi' : svi,
             'reg_lambda': reg_lambda,
             'prior_norm': prior_norm,             
             'round_cl' : round_cl,     
             
             'obs_stats' : obs_stats,
             'true_params' : true_params,

             'logs' : logs, 
             'logs_k' : logs_k, 
             'logs_k2' : logs_k2, 
             'tds' : tds,
             'tds_k' : tds_k,
             'tds_k2' : tds_k2,
             'posteriors' : posteriors,
             'posteriors_k' : posteriors_k,
             'posteriors_k2' : posteriors_k2
             })

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

seeds = np.arange(90, 110)
duration = 300


batch_specifier = '_bp'
a_ = 0.5
fit_specifier = '_pilot_convT_small_05'


for seed in seeds:
    true_params, labels_params = utils.obs_params(a=a_)
    obs = utils.obs_data(true_params, seed=seed, duration = duration)
    obs_stats = utils.obs_stats(true_params, seed=seed, duration = duration)

    rerun = False  # if False, will try loading file from disk

    try:
        assert rerun == False, 'rerun requested'
        sam = np.load('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz')['arr_0']
    except:
        sam = utils.pg_mcmc(true_params, obs, duration = duration)
        np.savez('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz', sam)

    n_train = 3000
    n_rounds = 10
    minibatch = 100
    epochs = 500
    round_cl = 7
    
    epochs_cbk = 5000
    minibatch_cbk = n_train
    
    n_hiddens = [5] 
    convert_to_T = 3
    pilot_samples = 1000
    svi = True
    reg_lambda = 0.01
    prior_norm = False
    
    if batch_specifier == '_bp':
        smoothing_prior = utils.smoothing_prior_highVar
    elif batch_specifier == '_cp':
        smoothing_prior = utils.smoothing_prior    
    
    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)

    res = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs, tds, posteriors = res.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss=None)

    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)
    res_k = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs_k, tds_k, posteriors_k = res_k.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss='x_kl')

    m = GLM(seed=seed, duration = duration)
    p = smoothing_prior(n_params=m.n_params, seed=seed)
    s = GLMStats(n_summary=m.n_params)
    g = dg.Default(model=m, prior=p, summary=s)
    res_k2 = infer.SNPE(g, 
                     obs=obs_stats, 
                     n_hiddens=n_hiddens, 
                     seed=seed, 
                     convert_to_T=convert_to_T, 
                     pilot_samples=pilot_samples,
                     svi=svi,
                     reg_lambda=reg_lambda,
                     prior_norm=prior_norm)

    logs_k2, tds_k2, posteriors_k2 = res_k2.run(n_train=n_train, 
                                    n_rounds=n_rounds, 
                                    minibatch=minibatch, 
                                    epochs=epochs, 
                                    round_cl=round_cl, 
                                    epochs_cbk=epochs_cbk,
                                    minibatch_cbk=minibatch_cbk,
                                    kernel_loss='ess')
    
    np.save('check_kernels' + batch_specifier + fit_specifier + '_d' + str(duration) + '_' + str(seed),
            {'seed': seed,
             'duration' : duration, 
             'n_train' : n_train,
             'n_rounds' : n_rounds,
             'minibatch' : minibatch,
             'epochs' : minibatch,

             'epochs_cbk' : epochs_cbk,
             'minibatch_cbk' : minibatch_cbk,
             
             'n_hiddens' : n_hiddens, 
             'convert_to_T' : convert_to_T, 
             'pilot_samples' : pilot_samples,
             'svi' : svi,
             'reg_lambda': reg_lambda,
             'prior_norm': prior_norm,             
             'round_cl' : round_cl,     
             
             'obs_stats' : obs_stats,
             'true_params' : true_params,

             'logs' : logs, 
             'logs_k' : logs_k, 
             'logs_k2' : logs_k2, 
             'tds' : tds,
             'tds_k' : tds_k,
             'tds_k2' : tds_k2,
             'posteriors' : posteriors,
             'posteriors_k' : posteriors_k,
             'posteriors_k2' : posteriors_k2
             })

In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import lfimodels.glm.utils as utils
import matplotlib.pyplot as plt
import numpy as np

from lfimodels.glm.GLM import GLM
from lfimodels.glm.GLMStats import GLMStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

seeds = np.arange(90, 110)
duration = 300


batch_specifier = '_bp'
a_ = 0.5
fit_specifier = '_pilot_convT_small_05'

if batch_specifier == '_bp':
    smoothing_prior = utils.smoothing_prior_highVar
elif batch_specifier == '_cp':
    smoothing_prior = utils.smoothing_prior    
    
for seed in seeds:
    true_params, labels_params = utils.obs_params(a=a_)
    obs = utils.obs_data(true_params, seed=seed, duration = duration)
    obs_stats = utils.obs_stats(true_params, seed=seed, duration = duration)

    rerun = True  # if False, will try loading file from disk

    try:
        assert rerun == False, 'rerun requested'
        sam = np.load('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz')['arr_0']
    except:
        sam = utils.pg_mcmc(true_params, obs, duration = duration, prior=smoothing_prior)
        np.savez('sam_' + str(duration) + '_' + str(seed) + batch_specifier + '.npz', sam)


In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             figsize=(9,9));

In [ ]:
# run with Gaussian proposals
for r in range(len(tds_k)):
    plot_pdf(posteriors[r],
             pdf2=posteriors_k2[r], 
             lims=[-2,2], 
             samples=sam, 
             gt=true_params, 
             figsize=(9,9));